<a href="https://colab.research.google.com/github/YoungsikMoon/FORS/blob/main/%EC%A0%95%ED%98%B8%EC%84%9D/FORS_%EC%A0%95%ED%98%B8%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 위스퍼파인튜닝_코랩용

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# 압축 풀 경로로 이동
%cd /content/drive/MyDrive/whisper/[원천]3.스튜디오
!pwd

In [ ]:
# 해당 경로에 압축 풀기 (압축파일 경로 찍어줌)
# 압축파일을 먼저 올려놓고 여기서 풀어주는게 훨씬 빠름

# 잘 못 누르면 데이터 꼬이니까 꼭 주석처리 해둘 것
# !unzip -qq "/content/drive/MyDrive/whisper/[원천]3.스튜디오.zip"

In [ ]:
# 필수 패키지 설치

!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install evaluate>=0.30
!pip install jiwer
!pip install accelerate -U
!pip install transformers[torch]

## 오디오 데이터셋 베이스 정보 리스트 만들기


- 필수 정보 : 오디오 데이터 파일 경로, 발화 텍스트


---


- 추가 정보 : json 파일 경로, 오디오 시간 정보
- 최종 데이터셋으로 만들기 전 단계
- 데이터 검증 및 필수 정보를 담은 리스트 추출 단계
- json 파일 열어서 내용 가져오는게 너무 오래 걸려서 로컬에서 만들어와서 편집하기로함


In [ ]:
# 오디오파일 경로 + 텍스트 매칭한 리스트 만들기

# 폴더와 파일 리스트를 뽑는데 유용한 라이브러리
# os 패키지로도 동일하게 수행 가능하지만 파일 리스트를 편하게 뽑는데 좀 더 특화돼있음
# 특히 glob 패키지의 glob 함수는 하위 폴더명이 아닌 하위 폴더의 전체 경로를 리스트로 반환해줘서 아주 편함
from glob import glob
from tqdm import tqdm
import json


# 문제는 폴더가 너무 많고 파일명이 복잡하다는 점
# 오디오파일과 json 파일 매칭이 제대로 되지 않을 경우 학습의미가 없음
    # ==> 데이터 프레임 만들 때 정확히 파일 이름이 일치하는 경우에만 매칭되도록 전처리 필요

# 오디오 파일 최상위 경로
# 폴더명, 파일명에 대괄호 들어있으면 인식 못함.. 수정할 것(수정 어려우면 os.listdir 사용)
audio_file_path = "/content/drive/MyDrive/whisper/원천_3_스튜디오/*"

# json 파일 최상위 경로
json_file_path = "/content/drive/MyDrive/whisper/라벨_3_스튜디오/*"


# 오디오 파일 구조 : [원천]3.스튜디오(1ea) - 노인남여_xxx폴더s(16ea) - 노인남녀_xxx.wav(다수)
# json 파일 구조 : [라벨]3.스튜디오(1ea) - 노인남여_xxx폴더s(16ea) - 노인남녀_xxx.json(다수)

# 원천, 라벨의 모든 폴더와 파일 이름이 같고, 같은 순서로 정렬됐는지 무결성 검사를 먼저 진행
# 진행하면서 파일 경로 리스트를 먼저 만들어둠

# 폴더 경로 리스트 정렬
audio_folder_path_list = sorted(glob(audio_file_path))
json_folder_path_list = sorted(glob(json_file_path))

# 폴더, 파일 갯수 매칭 확인부터. 갯수 다르면 다른 작업 시작할 필요가 없음.
# 갯수 하나라도 다르면 데이터 사용 불가.
if len(audio_folder_path_list) == len(json_folder_path_list):
    for audio_folder_path, json_folder_path in zip(audio_folder_path_list, json_folder_path_list):

        # 해당 폴더에 대한 파일 갯수가 같다면 계속 진행
        if len(glob(audio_folder_path + "/*")) == len(glob(json_folder_path + "/*")):
            continue

        # 다르면 에러 발생
        else:
            print(f"파일 갯수 다름 : {audio_folder_path}, {json_folder_path}")
            raise Exception("파일 무결성 검증 실패")

    print("데이터 갯수 무결성 검증 완료")

else:
    print("폴더 갯수 다름")
    raise Exception("파일 무결성 검증 실패")

# 갯수 검증 끝났으면 다음 단계 (폴더, 파일 이름 확인 후 경로 정보 리스트로 추출)
# 오디오, json 파일 경로 리스트
audio_file_path_list = []
json_file_path_list = []


# 첫 번째 루프 : 각 폴더 경로 꺼내옴
for audio_folder_path, json_folder_path in zip(audio_folder_path_list, json_folder_path_list):

    # 정렬된 파일 리스트 만들기
    audio_file_path_list_sorted = sorted(glob(audio_folder_path + "/*"))
    json_file_path_list_sorted = sorted(glob(json_folder_path + "/*"))

    # 두 번째 루프 : 각 파일
    for audio_file_path, json_file_path in zip(audio_file_path_list_sorted, json_file_path_list_sorted):

        # 확장자 제외 파일 이름이 같은지 확인
        if audio_file_path.split("/")[-1].split(".")[0] == json_file_path.split("/")[-1].split(".")[0]:

            # 이름 같으면 해당 파일 경로를 각 리스트에 넣어줌
            audio_file_path_list.append(audio_file_path)
            json_file_path_list.append(json_file_path)

        # 이름이 다르다면 데이터 매칭이 꼬여있다는 의미이므로 에러 발생 후 데이터 확인 필요
        else:
            print(f"파일 이름 불일치 : {audio_file_path}, {json_file_path}")
            raise Exception("파일 무결성 검증 실패")

print(f"폴더 및 파일 이름 매칭 검증 및 리스트 생성 완료, 갯수 : {len(audio_file_path_list)} : {len(json_file_path_list)}")


# 완성된 리스트에서 json 파일 안에 text 정보만 빼서 새로운 리스트 생성
# 녹음시간 통계를 보고 싶어서 시간정보도 추출
text_list = []
time_list = []

for json_file_path in json_file_path_list:
    with open (json_file_path, "r") as f:
        dict = json.load(f)
        text_list.append(dict["발화정보"]["stt"])
        time_list.append(dict["발화정보"]["recrdTime"])

# 갯수 확인
print(len(text_list), len(time_list))

데이터 갯수 무결성 검증 완료
폴더 및 파일 이름 매칭 검증 및 리스트 생성 완료, 갯수 : 0 : 0
0 0


#### - 최종 데이터 프레임 생성
- 코랩 json 읽기 느려서 로컬에서 작업한 파일 가져와서 편집
- 오디오 경로, json 경로, 발화텍스트, 녹음시간

In [ ]:
# 로컬에서 작업한 내용 코랩용으로 만들어서 다시 저장해주기
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/whisper/audio_path.csv", encoding = "utf-8")

# 루트 위치 바꿔주기
df["audio_path"] = df["audio_path"].str.replace("E:\\02.공부\\02.코딩\\01.Python\\01.Alphaco\\02.코드\\03.프로젝트\\02.자연어처리(음성인식)\\02.위스퍼파인튜닝\\01.data\\01.노인발화자유음성(스튜디오3)\\", "/content/drive/MyDrive/whisper/")
# 로컬에서는 \\, 코랩에서는 /로 구분. 바꿔주기
df["audio_path"] = df["audio_path"].str.replace("\\", "/")

# 새로 저장
df.to_csv("/content/drive/MyDrive/whisper/audio_path_colab.csv", index = False)

#### - 데이터 프레임 불러오기 및 녹음파일 테스트

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/whisper/audio_path_colab.csv", encoding = "utf-8")

# 경로 잘 되는지 테스트
from IPython.display import Audio
Audio(df.iloc[0, 0], rate=16000)

## DatasetDict 데이터셋 만들기



- 위에서 만들어진 "오디오파일 경로", "발화 텍스트" 리스트를 이용해 최종 자료형 생성

- DatasetDict 자료형이 위스퍼 최종 학습을 위한 자료형으로 사용됨
- 위스퍼 인풋 음성 파일의 Sample rate는 16khz로 고정

- Train, Vaildation, Test 세 개로 나눠진 최종 DatasetDict 자료형 생성


In [ ]:
# 이제부터 시작..

# 뽑은 파일 경로 데이터를 위스퍼가 요구하는 데이터셋 형태로 변환
# 최종적으로 DatasetDict 객체로 만들어줘야함
from datasets import Dataset, DatasetDict

# 오디오 파일의 sampling rate를 조절해줌
# 위스퍼에서는 무조건 16khz(16,000hz)로 맞춰줘야함.
from datasets import Audio

# 오디오파일 경로와 텍스트를 딕셔너리 형태로 넣어줌
# Dataset에서 제공하는 cast_column 함수에서 audio의 value값에 Audio함수를 적용해 sampling_rate를 조절해줌 (cast_column()은 일종의 map 같은 함수인듯)
ds = Dataset.from_dict({"audio" : df["audio_path"].tolist(), "transcripts" : df["text"].tolist()}).cast_column("audio", Audio(sampling_rate = 16000))

# 위의 ds는 전체 데이터
# 전체 중에서 validation 데이터와 test 데이터를 분할해줌

# 훈련데이터(0.8), validation(0.2) 비율로 나눠줌
ds_train_valid = ds.train_test_split(test_size = 0.2)

# validation 중에서 절반을 다시 테스트 데이터로 나눠줌
# 데이터셋은 딕셔너리 형태로 데이터를 쪼개서 보관하니까 컬럼명으로 찾아주면 됨 (train, test)
ds_test = ds_train_valid["test"].train_test_split(test_size = 0.5)

# train, validation, test 세 가지로 쪼개진 데이터를 DatasetDict 형태로 한 번에 묶어주면 데이터 가공 끝
# 구조가 약간 헷갈릴 수 있음
    # 처음 만든 ds_train_valid의 ["train"]은 0.8의 훈련 데이터를 의미
    # ds_train_valid의 0.2인 ["test"]를 다시 반반씩 쪼개서 하나는 validation, 하나는 test로 사용하기로 함
        # 따라서 ds_test의 ["train"]과 ["test"]가 각각 validation, test 자료가 되는 구조 (비율이 반반이니까 순서는 상관없음)
datasets = DatasetDict({"train" : ds_train_valid["train"], "valid" : ds_test["train"], "test" : ds_test["test"]})

# 허깅페이스에 업로드해서 보관하는 데이터셋은 위의 datasets임.
# 구글드라이브에는 이 객체 자체를 저장 못하니까 허깅페이스에서 지원해주는듯
# 나중에 체크포인트 등도 지원해주니까 업로드해서 쓰는게 좋을 것 같음

datasets["train"]

Dataset({
    features: ['audio', 'transcripts'],
    num_rows: 13927
})

#### 전처리 완료된 최종 학습용 datasetDict 만들기
- 패딩, mel-log로 변환된 input_features 컬럼 / 인덱스로 변환된 labels 컬럼


---

- 기존 컬럼 (audio, transcripts)는 버리고 전처리 완료된 datasetDict 객체를 생성
- 전처리 작업 약 40분 넘게 소요 예정. 객체 저장 필수
- 허깅 페이스는 매 번 토큰 입력해줘야함으로 구글 드라이브에 저장
- 코랩 메모리 아웃나서 로컬에서 진행..=ㅅ=
    - 로컬에서는 10분 걸림..-ㅅ-
    - 코랩프로 결제해도 2시간 걸리는데..? 이게 맞는거임..???
    - 아무래도 구글 드라이브를 쓰다보니 I/O가 엄청나게 느린듯..ㅅㅂ..
    - 전처리는 로컬이 짱인듯.. 근데 이건 또 드라이브에 업로드하는 시간이...ㅅㅂ..
    - GPU 빵빵하게 달린 서버 한 대 쓰는게 최고다.. 코랩프로+도 그닥 소용없을 듯..

In [ ]:
# 오디오파일 전처리(학습전), 후처리(학습후) 해주는 클래스
    # WhisperFeatureExtractor 클래스와 WhisperTokenizer 클래스를 상속받아 하나로 쓸 수 있도록 합쳐둔 클래스
    # 위 두 개 클래스는 따로 import 하지 않아도 됨
from transformers import WhisperProcessor

# 객체 생성
# 파라미터 : 베이스 모델(tiny, small, medium 등), 언어종류, 태스크
    # transcribe는 해당 언어로 전사를 뜻함. 바로 영어로 번역하려면 다른 태스크 모드(translate) 사용
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")


# 데이터 전처리 작업을 위한 함수
def prepare_dataset(batch):
    # 오디오 파일을 16kHz로 로드
    audio = batch["audio"]

    # input audio array로부터 sample_rate값 적용해서 log-Mel spectrogram 변환 및 30초로 패딩작업
    # 오디오가 다 짧은데 꼭 30초로 패딩해야하는가에 대한 의문. 줄이는 방법이 안보임. 못줄인다네..-ㅅ-
    # input_features[0] : input_features, input_features[1] : attention_mask (위스퍼에서는 30초로 일괄 패딩해버리니까 불필요)
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate = audio["sampling_rate"]).input_features[0]

    # target text를 label ids로 변환
    batch["labels"] = processor.tokenizer(batch["transcripts"]).input_ids
    return batch


# 데이터 전처리 함수를 데이터셋 전체에 적용
# num_proc : 몇 개의 CPU 코어를 사용할 지 결정 (2개부터 병렬처리)
    # 코랩 CPU 코어 확인 : !grep "cpu cores" /proc/cpuinfo | tail -1 ==> 1코어밖에 안줌..

# remove_columns는 반환할 때 해당 컬럼은 제외한 객체를 반환한다는 의미
    # datasets.column_names["train"] : ["audio", "transcripts"] (오디오, 발화텍스트)
    # 즉, "audio"에 들어있는 path, array, sampling_rate 정보를 써서 실제 log-Mel로 전처리된 "input_features"를 생성
    # "transcripts"에 들어있는 한국어는 인덱스로 변환시켜 전처리된 "labels"를 생성
        # 전처리된 데이터를 가진 "input_features"와 "labels"만 가진 새로운 datasetDict 타입의 객체를 반환 (나머지 컬럼은 제거된)
        # 굳이 remove_columns() 옵션을 사용하지 않고 그냥 함수 내에서 작업 후 해당 컬럼을 지워버려도 무방할 듯

low_call_voices = datasets.map(prepare_dataset, remove_columns = datasets.column_names["train"], num_proc=1)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/13927 [00:00<?, ? examples/s]

Map:   0%|          | 0/1741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1741 [00:00<?, ? examples/s]

#### - 최종 전처리된 datasetDict 객체 저장하고 불러오기

> 인용구 추가


- 이제 데이터 전처리는 끝
- 이 객체만 있으면 코랩 말고 다른 곳에서도 학습시킬 수 있음
- 이후 프로세스는 TPU 또는 GPU가 필요
    - 근데 GPU가 없네.. GPU 내놔라...=ㅅ=

- 문제는 최종 전처리된 객체가 3G -> 16G로 용량이 늘어났다는 것..
    - 5~10초짜리 음성파일이 대부분인데 30초로 일괄 패딩해버린 탓..
    - 패딩 길이 조절하는 방법은 없다고함
        - 그냥 30초 내외 음성파일 쓰는게 가장 효율적일 듯

In [ ]:
# 전처리 완료된 datasetDict 객체 디스크(구글 드라이브)에 저장하기 (폴더로 저장됨)
    # 드라이브에 15기가 공간 없음. 그냥 content에 저장..
# 실제로는 수행안함. 로컬에서 해서 드라이브에 가져다둠. 딥빡..
low_call_voices.save_to_disk("/content/drive/MyDrive/whisper/low_call_voice")

Saving the dataset (0/27 shards):   0%|          | 0/13927 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/1741 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/1741 [00:00<?, ? examples/s]

In [ ]:
# 전처리 완료된 datasetDict 객체 디스크(구글 드라이브)에서 불러오기
from datasets import load_from_disk
from datasets import Dataset, DatasetDict
from transformers import WhisperProcessor

# 여기서부터 새로 시작.
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")
low_call_voices_prepreocessed = load_from_disk("/content/drive/MyDrive/whisper/low_call_voice")
low_call_voices_prepreocessed

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading dataset from disk:   0%|          | 0/27 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 13927
    })
    valid: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1741
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1741
    })
})

## 데이터 학습 (Fine-Tuning) 진행


- 전처리된 데이터로 학습진행 시작
- 여기서부터 GPU/TPU 필수
- GPU/TPU 주는 캐글 노트북으로 건너가서 진행
    - 아래 코드는 코랩에서는 캐글에서 수행할 것..
    - 근데 캐글에 16기가 파일 안올라가서 실패..

#### 1. Data Collator
- 전처리한 데이터를 모델에 입력할 수 있는 PyTorch 텐서 형태로 변환하는 작업
- 이 때 패딩이 되지 않은 타겟(labels = 발화데이터)도 같이 패딩처리해줌

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import WhisperProcessor

# 전처리된 데이터를 텐서 형태로 변환
# 제일 이해 안가는 코드..

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:

    processor: Any

    def __call__(self, features : List[Dict[str, Union[List[int], torch.Tednsor]]]) -> Dict[str, torch.Tensor]:

        # featurs는 최종 datasetDict 자료 (3개로 이뤄져있음)
            # train : input_features, labels
            # valid : input_features, labels
            # test : input_features, labels

        # 인풋 데이터(mel-log로 변환된 오디오 파일)를 토치 텐서로 변환
        input_features = [{"input_features": feature["input_features"]} for feature in features]

        # input_feaures를 패딩. 이미 패딩 돼 있는데 왜 또하는지는 잘 모르겠음.
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # 라벨 데이터 (정수 인코딩된 발화데이터)를 토치 텐서로 변환
        # ex) train이 가진 ["input_features"] 컬럼을 딕셔너리 구조를 가진 리스트 안에다가 넣어줌
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # 라벨 데이터 패딩 적용
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 패딩 토큰을 -100으로 치환해 loss 계산 과정에서 무시되도록 함
        # 이미 패딩 토큰이라는 것 자체가 무시되도록 돼 있을텐데 왜 하는지를 잘 모르겠음
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라냄
        # 이 코드는 뭔지 잘 모르겠음..
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


# 데이터 콜레이터 초기화
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

#### 2. Evaluaion Metrics
- 검증 데이터셋에 사용할 검증 매트릭스 정의
- 한국어의 경우 WER 보다 CER이 더 적합

In [ ]:
import evaluate

metric = evaluate.load('cer')

def compute_metrics(pred):

    # 예측값
    pred_ids = pred.predictions

    # 실제값
    label_ids = pred.label_ids

    # 패딩된 토큰을 올바르게 무시하기 위해 -100을 다시 pad_token으로 치환
    # 밑에 kip_special_tokens=True가 적용될 수 있도록 함
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # 정수 인덱스를 실제 문자열로 디코딩
    # metrics 계산 시 special token들을 빼고 계산하도록 설정
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # CER 계산
    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

#### 3. Load a Pre-Trained Checkpoint
- pre_trained model : 이미 학습된 위스퍼의 기본 모델을 뜻함
- tiny, small, medium, large 등의 학습된 모델 종류를 지정해서 모델을 로드

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

# 한국어 고정이기 때문에 언어를 고정해주는 것이 좋음
model.generation_config.language = "korean"

# 한국어 전사 태스크임을 명시
# 프로세서에서 명시해줬지만 모델에서도 다시 명시해주는게 좋은 듯
model.generation_config.task = "transcribe"

# 디폴트값인 any는 디코더가 다국어에 맞는 토큰을 자동으로 찾도록 함
# 한국어만 보면 되므로 None으로 지정해서 좀 더 정확성을 올려줌
model.config.forced_decoder_ids = None

# 문장 생성 중 억제되는 토큰이 없도록 리스트를 비워줌
# 정확한 의미는 모르겠음..
# suppress_tokens는 일종의 불용어 사전 같은 용도
model.config.suppress_tokens = []

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

####4. Define the Training Arguments
- 최종 학습을 위한 파라미터 설정
- 에포크 횟수, 모델 저장 경로 등등

In [ ]:
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer


# 파라미터 설정
# 너무 많다..
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/whisper/save_model",  # 모델의 최종 결과(가중치 테이블)이 저장될 곳
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=500,
    num_train_epochs = 10,
    # max_steps=4000,  # epoch 대신 설정, Test할 때만 사용하는 파라미터
    gradient_checkpointing=True,
    fp16=True, # Cuda 사용 못하면 False로 둬야함
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합
    greater_is_better=False,
    push_to_hub=False, # 허브에 업로드할지 여부
)

# 트레이너 설정
# 위에서 나온 모든 객체들을 이용해서 트레이닝
trainer = Seq2SeqTrainer(
    args=training_args, # 위에서 설정한 파라미터 적용
    model=model,
    train_dataset=low_call_voices_prepreocessed["train"],
    eval_dataset=low_call_voices_prepreocessed["valid"],  # or "test"
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1463: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


####5. 학습 진행
- 학습 진행
- 진행 후 바로 저장
- 리소스가 있어야 하지..

In [ ]:
# GPU 탕진잼!!
trainer.train()

# 모델, 프로세서 저장 (토크나이저는 프로세서 안에 있으니까 따로 저장안함)
trainer.save_pretrained("/content/drive/MyDrive/whisper/save_model/")
processor.save_pretrained("/content/drive/MyDrive/whisper/save_model/")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Cer
1000,0.064200,0.096782,3.302736
2000,0.014900,0.048441,1.866198
3000,0.007600,0.041185,1.549378
4000,0.002400,0.036495,1.484278
5000,0.001500,0.035855,1.466918
6000,0.000800,0.035021,1.347569


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/do

Step,Training Loss,Validation Loss,Cer
1000,0.064200,0.096782,3.302736
2000,0.014900,0.048441,1.866198
3000,0.007600,0.041185,1.549378
4000,0.002400,0.036495,1.484278
5000,0.001500,0.035855,1.466918
6000,0.000800,0.035021,1.347569


## 평가진행



- 위 과정은 train, vaild를 이용한 훈련과 검증 과정
- 평가는 Test를 사용해 진행

In [ ]:
from transformers import Seq2SeqTrainer

# 저장한 모델과 프로세서 로드
model = WhisperForConditionalGeneration.from_pretrained("/content/drive/MyDrive/whisper/save_model/")
processor = WhisperProcessor.from_pretrained("/content/drive/MyDrive/whisper/save_model/")

In [ ]:
# 트레이너 셋팅. training_args는 훈련에서 사용한 것과 동일하게 사용
# 여기서 train은 왜 집어넣는걸까..? 필요없는데..
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=low_call_voices_prepreocessed["train"],
    eval_dataset=low_call_voices_prepreocessed["test"],  # for evaluation(not validation)
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# 평가 진행
trainer.evaluate()

## 모델 사용해보기

In [ ]:
from transformers import pipeline
import gradio as gr

# 저장된 모델 불러와서 파이프라인에 넣어주면 됨
pipe = pipeline(model="")

# 실제 사용 메소드는 이거 3줄이면 됨. 물론 오디오 전처리 코드는 따로.
def transcribe(audio):

    # 파이프라인에 오디오파일 넣어주면 모델이 텍스트로 변환해서 반환해줌
    text = pipe(audio)["text"]
    return text

# 녹음하자마자 음성 파일로 따서 모델에 전달하고 싶다면 그라디오 사용
iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()